# 震荡指标

## 📋 概述

震荡指标（Oscillators）是技术分析中的重要工具，用于判断市场的超买超卖状态。当价格在某个区间内震荡时，震荡指标可以帮助识别价格的转折点。本章节将详细介绍常用的震荡指标，包括RSI、KDJ、威廉指标等。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际图表观察震荡指标的效果，加深理解。


## 📦 环境准备

首先安装必要的Python库：


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib mplfinance akshare

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import akshare as ak
from datetime import datetime, timedelta

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. RSI指标（相对强弱指标）

RSI是最常用的震荡指标之一，用于衡量价格变动的速度和幅度，判断市场的超买超卖状态。

#### 1.1 RSI的计算

**计算公式**：
$$RSI(n) = 100 - \frac{100}{1 + RS}$$

其中：
- $RS = \frac{平均上涨幅度}{平均下跌幅度}$
- $n$ 通常为14

**信号含义**：
- **RSI > 70**：超买区域，可能下跌，考虑卖出
- **RSI < 30**：超卖区域，可能上涨，考虑买入
- **RSI = 50**：中性区域

让我们通过代码计算和绘制RSI指标：


In [ ]:
# 获取股票数据
try:
    stock_data = ak.stock_zh_a_hist(symbol="000001", period="daily", 
                                     start_date="20240101", end_date="20241231", 
                                     adjust="qfq")
    
    stock_data.columns = ['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率']
    stock_data['日期'] = pd.to_datetime(stock_data['日期'])
    stock_data = stock_data.set_index('日期')
    stock_data = stock_data[['开盘', '最高', '最低', '收盘', '成交量']]
    stock_data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    
    print("数据获取成功！")
    
except Exception as e:
    print(f"数据获取失败：{e}")
    print("使用模拟数据演示...")
    dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
    dates = dates[dates.weekday < 5]
    
    np.random.seed(42)
    prices = 10 + np.cumsum(np.random.randn(len(dates)) * 0.1)
    
    stock_data = pd.DataFrame({
        'Open': prices + np.random.randn(len(dates)) * 0.05,
        'High': prices + np.abs(np.random.randn(len(dates)) * 0.1),
        'Low': prices - np.abs(np.random.randn(len(dates)) * 0.1),
        'Close': prices + np.random.randn(len(dates)) * 0.05,
        'Volume': np.random.randint(1000000, 10000000, len(dates))
    }, index=dates)
    
    stock_data['High'] = stock_data[['Open', 'Close', 'High']].max(axis=1)
    stock_data['Low'] = stock_data[['Open', 'Close', 'Low']].min(axis=1)
    
    print("使用模拟数据演示")


In [ ]:
# 计算RSI指标
def calculate_rsi(data, period=14):
    """计算RSI指标"""
    # 计算价格变化
    delta = data.diff()
    
    # 分离上涨和下跌
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    # 计算RS和RSI
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# 计算RSI
stock_data['RSI'] = calculate_rsi(stock_data['Close'])

# 绘制K线图和RSI
recent_data = stock_data.tail(100)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True,
                                gridspec_kw={'height_ratios': [3, 1]})

# 上图：K线图
mpf.plot(recent_data[['Open', 'High', 'Low', 'Close', 'Volume']], 
         type='candle', style='charles', ax=ax1, volume=False)
ax1.set_title('K线图与RSI指标', fontsize=16, fontweight='bold')
ax1.set_ylabel('价格')
ax1.grid(True, alpha=0.3)

# 下图：RSI指标
ax2.plot(recent_data.index, recent_data['RSI'], label='RSI', linewidth=1.5, color='blue')
ax2.axhline(y=70, color='red', linestyle='--', linewidth=1, label='超买线(70)')
ax2.axhline(y=30, color='green', linestyle='--', linewidth=1, label='超卖线(30)')
ax2.axhline(y=50, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
ax2.fill_between(recent_data.index, 70, 100, alpha=0.1, color='red', label='超买区域')
ax2.fill_between(recent_data.index, 0, 30, alpha=0.1, color='green', label='超卖区域')
ax2.set_title('RSI指标', fontsize=14, fontweight='bold')
ax2.set_xlabel('日期')
ax2.set_ylabel('RSI值')
ax2.set_ylim(0, 100)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 分析RSI信号
recent_data['超买'] = recent_data['RSI'] > 70
recent_data['超卖'] = recent_data['RSI'] < 30

print("RSI分析：")
print(f"当前RSI值：{recent_data['RSI'].iloc[-1]:.2f}")
print(f"超买次数：{recent_data['超买'].sum()}")
print(f"超卖次数：{recent_data['超卖'].sum()}")

if recent_data['RSI'].iloc[-1] > 70:
    print("⚠️ 当前处于超买区域，可能下跌")
elif recent_data['RSI'].iloc[-1] < 30:
    print("✅ 当前处于超卖区域，可能上涨")
else:
    print("📊 当前处于中性区域")


### 2. KDJ指标

KDJ指标是随机指标，结合了最高价、最低价和收盘价，用于判断超买超卖和买卖时机。

#### 2.1 KDJ的计算

**计算公式**：
1. **RSV（未成熟随机值）**：
   $$RSV(n) = \frac{Close - Low_n}{High_n - Low_n} \times 100$$
   
2. **K值**：RSV的m日移动平均
   $$K = \frac{2}{3} \times K_{前} + \frac{1}{3} \times RSV$$

3. **D值**：K值的m日移动平均
   $$D = \frac{2}{3} \times D_{前} + \frac{1}{3} \times K$$

4. **J值**：
   $$J = 3 \times K - 2 \times D$$

其中：
- $n$ 通常为9（周期）
- $m$ 通常为3（平滑周期）

**信号含义**：
- **K > 80, D > 80**：超买区域，考虑卖出
- **K < 20, D < 20**：超卖区域，考虑买入
- **K上穿D**：买入信号（金叉）
- **K下穿D**：卖出信号（死叉）

让我们计算和绘制KDJ指标：


In [ ]:
# 计算KDJ指标
def calculate_kdj(data, high_col='High', low_col='Low', close_col='Close', n=9, m=3):
    """计算KDJ指标"""
    # 计算RSV
    low_n = data[low_col].rolling(window=n).min()
    high_n = data[high_col].rolling(window=n).max()
    rsv = (data[close_col] - low_n) / (high_n - low_n) * 100
    rsv = rsv.fillna(0)
    
    # 计算K值（初始值为50）
    k = pd.Series(index=data.index, dtype=float)
    k.iloc[0] = 50
    for i in range(1, len(data)):
        k.iloc[i] = (2/3) * k.iloc[i-1] + (1/3) * rsv.iloc[i]
    
    # 计算D值（初始值为50）
    d = pd.Series(index=data.index, dtype=float)
    d.iloc[0] = 50
    for i in range(1, len(data)):
        d.iloc[i] = (2/3) * d.iloc[i-1] + (1/3) * k.iloc[i]
    
    # 计算J值
    j = 3 * k - 2 * d
    
    return k, d, j

# 计算KDJ
stock_data['K'], stock_data['D'], stock_data['J'] = calculate_kdj(stock_data)

# 绘制K线图和KDJ
recent_data = stock_data.tail(100)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True,
                                gridspec_kw={'height_ratios': [3, 1]})

# 上图：K线图
mpf.plot(recent_data[['Open', 'High', 'Low', 'Close', 'Volume']], 
         type='candle', style='charles', ax=ax1, volume=False)
ax1.set_title('K线图与KDJ指标', fontsize=16, fontweight='bold')
ax1.set_ylabel('价格')
ax1.grid(True, alpha=0.3)

# 下图：KDJ指标
ax2.plot(recent_data.index, recent_data['K'], label='K', linewidth=1.5, color='blue')
ax2.plot(recent_data.index, recent_data['D'], label='D', linewidth=1.5, color='red')
ax2.plot(recent_data.index, recent_data['J'], label='J', linewidth=1.5, color='green', linestyle='--')
ax2.axhline(y=80, color='red', linestyle='--', linewidth=1, label='超买线(80)')
ax2.axhline(y=20, color='green', linestyle='--', linewidth=1, label='超卖线(20)')
ax2.fill_between(recent_data.index, 80, 100, alpha=0.1, color='red', label='超买区域')
ax2.fill_between(recent_data.index, 0, 20, alpha=0.1, color='green', label='超卖区域')
ax2.set_title('KDJ指标', fontsize=14, fontweight='bold')
ax2.set_xlabel('日期')
ax2.set_ylabel('KDJ值')
ax2.set_ylim(0, 100)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 识别金叉和死叉
recent_data['KDJ_金叉'] = (recent_data['K'] > recent_data['D']) & (recent_data['K'].shift(1) <= recent_data['D'].shift(1))
recent_data['KDJ_死叉'] = (recent_data['K'] < recent_data['D']) & (recent_data['K'].shift(1) >= recent_data['D'].shift(1))

golden_cross = recent_data[recent_data['KDJ_金叉']]
death_cross = recent_data[recent_data['KDJ_死叉']]

print("KDJ分析：")
print(f"当前K值：{recent_data['K'].iloc[-1]:.2f}")
print(f"当前D值：{recent_data['D'].iloc[-1]:.2f}")
print(f"当前J值：{recent_data['J'].iloc[-1]:.2f}")
print(f"\n识别到 {len(golden_cross)} 个金叉信号（买入）")
print(f"识别到 {len(death_cross)} 个死叉信号（卖出）")

if recent_data['K'].iloc[-1] > 80 and recent_data['D'].iloc[-1] > 80:
    print("⚠️ 当前处于超买区域")
elif recent_data['K'].iloc[-1] < 20 and recent_data['D'].iloc[-1] < 20:
    print("✅ 当前处于超卖区域")


### 3. 威廉指标（WR）

威廉指标用于衡量超买超卖状态，与RSI类似但计算方法不同。

#### 3.1 WR的计算

**计算公式**：
$$WR(n) = \frac{High_n - Close}{High_n - Low_n} \times (-100)$$

其中：
- $High_n$ 是n日内的最高价
- $Low_n$ 是n日内的最低价
- $n$ 通常为14

**信号含义**：
- **WR < -80**：超卖区域，可能上涨，考虑买入
- **WR > -20**：超买区域，可能下跌，考虑卖出

让我们计算和绘制威廉指标：


In [ ]:
# 计算威廉指标（WR）
def calculate_wr(data, high_col='High', low_col='Low', close_col='Close', period=14):
    """计算威廉指标"""
    high_n = data[high_col].rolling(window=period).max()
    low_n = data[low_col].rolling(window=period).min()
    wr = (high_n - data[close_col]) / (high_n - low_n) * (-100)
    return wr

# 计算WR
stock_data['WR'] = calculate_wr(stock_data)

# 绘制K线图和WR
recent_data = stock_data.tail(100)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True,
                                gridspec_kw={'height_ratios': [3, 1]})

# 上图：K线图
mpf.plot(recent_data[['Open', 'High', 'Low', 'Close', 'Volume']], 
         type='candle', style='charles', ax=ax1, volume=False)
ax1.set_title('K线图与威廉指标（WR）', fontsize=16, fontweight='bold')
ax1.set_ylabel('价格')
ax1.grid(True, alpha=0.3)

# 下图：WR指标
ax2.plot(recent_data.index, recent_data['WR'], label='WR', linewidth=1.5, color='purple')
ax2.axhline(y=-20, color='red', linestyle='--', linewidth=1, label='超买线(-20)')
ax2.axhline(y=-80, color='green', linestyle='--', linewidth=1, label='超卖线(-80)')
ax2.fill_between(recent_data.index, -20, 0, alpha=0.1, color='red', label='超买区域')
ax2.fill_between(recent_data.index, -100, -80, alpha=0.1, color='green', label='超卖区域')
ax2.set_title('威廉指标（WR）', fontsize=14, fontweight='bold')
ax2.set_xlabel('日期')
ax2.set_ylabel('WR值')
ax2.set_ylim(-100, 0)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 分析WR信号
recent_data['WR_超买'] = recent_data['WR'] > -20
recent_data['WR_超卖'] = recent_data['WR'] < -80

print("威廉指标（WR）分析：")
print(f"当前WR值：{recent_data['WR'].iloc[-1]:.2f}")
print(f"超买次数：{recent_data['WR_超买'].sum()}")
print(f"超卖次数：{recent_data['WR_超卖'].sum()}")

if recent_data['WR'].iloc[-1] > -20:
    print("⚠️ 当前处于超买区域，可能下跌")
elif recent_data['WR'].iloc[-1] < -80:
    print("✅ 当前处于超卖区域，可能上涨")
else:
    print("📊 当前处于中性区域")


### 4. 综合应用：多个震荡指标结合

在实际交易中，通常结合多个震荡指标来提高信号的可靠性。让我们同时观察RSI、KDJ和WR：


In [ ]:
# 综合多个震荡指标
recent_data = stock_data.tail(100)

fig, axes = plt.subplots(4, 1, figsize=(15, 16), sharex=True,
                          gridspec_kw={'height_ratios': [3, 1, 1, 1]})

# 第一个图：K线图
mpf.plot(recent_data[['Open', 'High', 'Low', 'Close', 'Volume']], 
         type='candle', style='charles', ax=axes[0], volume=False)
axes[0].set_title('K线图与多个震荡指标', fontsize=16, fontweight='bold')
axes[0].set_ylabel('价格')
axes[0].grid(True, alpha=0.3)

# 第二个图：RSI
axes[1].plot(recent_data.index, recent_data['RSI'], label='RSI', linewidth=1.5, color='blue')
axes[1].axhline(y=70, color='red', linestyle='--', linewidth=1)
axes[1].axhline(y=30, color='green', linestyle='--', linewidth=1)
axes[1].fill_between(recent_data.index, 70, 100, alpha=0.1, color='red')
axes[1].fill_between(recent_data.index, 0, 30, alpha=0.1, color='green')
axes[1].set_title('RSI指标', fontsize=12, fontweight='bold')
axes[1].set_ylabel('RSI')
axes[1].set_ylim(0, 100)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 第三个图：KDJ
axes[2].plot(recent_data.index, recent_data['K'], label='K', linewidth=1.5, color='blue')
axes[2].plot(recent_data.index, recent_data['D'], label='D', linewidth=1.5, color='red')
axes[2].plot(recent_data.index, recent_data['J'], label='J', linewidth=1.5, color='green', linestyle='--')
axes[2].axhline(y=80, color='red', linestyle='--', linewidth=1)
axes[2].axhline(y=20, color='green', linestyle='--', linewidth=1)
axes[2].fill_between(recent_data.index, 80, 100, alpha=0.1, color='red')
axes[2].fill_between(recent_data.index, 0, 20, alpha=0.1, color='green')
axes[2].set_title('KDJ指标', fontsize=12, fontweight='bold')
axes[2].set_ylabel('KDJ')
axes[2].set_ylim(0, 100)
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# 第四个图：WR
axes[3].plot(recent_data.index, recent_data['WR'], label='WR', linewidth=1.5, color='purple')
axes[3].axhline(y=-20, color='red', linestyle='--', linewidth=1)
axes[3].axhline(y=-80, color='green', linestyle='--', linewidth=1)
axes[3].fill_between(recent_data.index, -20, 0, alpha=0.1, color='red')
axes[3].fill_between(recent_data.index, -100, -80, alpha=0.1, color='green')
axes[3].set_title('威廉指标（WR）', fontsize=12, fontweight='bold')
axes[3].set_xlabel('日期')
axes[3].set_ylabel('WR')
axes[3].set_ylim(-100, 0)
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 综合信号分析
print("=" * 50)
print("综合震荡指标分析")
print("=" * 50)
print(f"当前RSI：{recent_data['RSI'].iloc[-1]:.2f}")
print(f"当前K值：{recent_data['K'].iloc[-1]:.2f}, D值：{recent_data['D'].iloc[-1]:.2f}")
print(f"当前WR：{recent_data['WR'].iloc[-1]:.2f}")

# 综合判断
buy_signals = 0
sell_signals = 0

if recent_data['RSI'].iloc[-1] < 30:
    buy_signals += 1
elif recent_data['RSI'].iloc[-1] > 70:
    sell_signals += 1

if recent_data['K'].iloc[-1] < 20 and recent_data['D'].iloc[-1] < 20:
    buy_signals += 1
elif recent_data['K'].iloc[-1] > 80 and recent_data['D'].iloc[-1] > 80:
    sell_signals += 1

if recent_data['WR'].iloc[-1] < -80:
    buy_signals += 1
elif recent_data['WR'].iloc[-1] > -20:
    sell_signals += 1

print(f"\n买入信号数：{buy_signals}/3")
print(f"卖出信号数：{sell_signals}/3")

if buy_signals >= 2:
    print("✅ 多个指标显示超卖，可能上涨")
elif sell_signals >= 2:
    print("⚠️ 多个指标显示超买，可能下跌")
else:
    print("📊 指标信号不一致，需要结合其他分析方法")
